In [2]:
import os
import sys
sys.path.append('../')
import math
import numpy as np
from src.dataset import EnergyDataset
from src.modeling.train import EnergyModelTrainer
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sktime.utils.plotting import plot_series
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

save_path = 'models'
os.makedirs(save_path, exist_ok=True)

/home/ansem/Desktop/ve2max_assessment/myenv/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-07-28 20:58:43.302665: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 20:58:43.366390: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 20:58:43.367217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-28 20:58:44.719599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dataset = EnergyDataset(data_path="../data/raw/data.csv")
dataset._perform_feature_engineering()
X_train, X_test, y_train, y_test = dataset._train_test_split()
trainer = EnergyModelTrainer(X_train, y_train)

['voltage', 'reactive_power', 'power_factor', 'temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity', 'speed', 'deg', 'active_power_lag_1', 'active_power_lag_8', 'year', 'month_of_year', 'week_of_year', 'day_of_year', 'day_of_week', 'hour_of_day', 'is_weekend', 'is_holiday', 'temp_mean_window_24', 'temp_std_window_24', 'temp_mean_window_168', 'temp_std_window_168', 'temp_mean_window_8760', 'temp_std_window_8760', 'feels_like_mean_window_24', 'feels_like_std_window_24', 'feels_like_mean_window_168', 'feels_like_std_window_168', 'feels_like_mean_window_8760', 'feels_like_std_window_8760', 'temp_min_mean_window_24', 'temp_min_std_window_24', 'temp_min_mean_window_168', 'temp_min_std_window_168', 'temp_min_mean_window_8760', 'temp_min_std_window_8760', 'temp_max_mean_window_24', 'temp_max_std_window_24', 'temp_max_mean_window_168', 'temp_max_std_window_168', 'temp_max_mean_window_8760', 'temp_max_std_window_8760', 'pressure_mean_window_24', 'pressure_std_window_24', 'pressure

In [ ]:
#arima_predictions, arima_training_time, y_test, dates = trainer.train_evaluate_arima()
#prophet_predictions, prophet_training_time = trainer.train_evaluate_prophet()
lstm_predictions, lstm_training_time = trainer.train_evaluate_lstm()

In [ ]:
def predict_test_lstm(X_test, model_path='../models/lstm_model_2.h5'):
      
      loaded_model = load_model(os.path.join(save_path, model_path))

      X_test = X_test.drop(columns=["date"])
      X_test_values = X_test.values

      timesteps = 1
      num_features = X_test_values.shape[1]
      X_test_reshaped = X_test_values.reshape((X_test_values.shape[0], timesteps, num_features))

      predictions = loaded_model.predict(X_test_reshaped).flatten()

      return predictions

In [ ]:
lstm_test_predictions = predict_test_lstm(X_test)

score_mae = mean_absolute_error(y_test, lstm_test_predictions)
score_rmse = math.sqrt(mean_squared_error(y_test, lstm_test_predictions))
print(f"LSTM Test Set: MAE: {score_mae:.4f}, RMSE: {score_rmse:.4f}")

In [ ]:
df_train = pd.DataFrame({
    'Date': dates,
    'y_train': y_test
})

df_predictions = pd.DataFrame({
    'Date': dates,
    'LSTM_Predictions': lstm_test_predictions
})

df_train.reset_index(drop=True, inplace=True)
df_predictions.reset_index(drop=True, inplace=True)

df_train['Date'] = pd.to_datetime(df_train['Date']).dt.date
df_predictions['Date'] = pd.to_datetime(df_predictions['Date']).dt.date

print(df_train.head())
print(df_predictions.head())

In [ ]:

def round_by_second_digit(value):
    value_shifted = value * 10
    value_rounded = np.round(value_shifted* 10) / 10
    return value_rounded

In [ ]:
df_predictions['LSTM_Predictions'] = df_predictions['LSTM_Predictions'].apply(round_by_second_digit)


In [ ]:
mean_predictions = df_predictions.groupby('Date')['LSTM_Predictions'].mean()
mean_actual = df_train.groupby('Date')['y_train'].mean()


mean_predictions.index = pd.to_datetime(mean_predictions.index)
mean_actual.index = pd.to_datetime(mean_actual.index)

plot_series(mean_predictions, mean_actual, colors=['red', 'blue'], labels=['Predictions', 'Actual'])
plt.show()

In [ ]:

summary_df = pd.DataFrame({
    'Model': [
              #'ARIMA'
              #, 'Prophet',
              'LSTM'
              ],
    'MAE': [
            #mean_absolute_error(y_test, arima_predictions),
            #mean_absolute_error(y_train, prophet_predictions),
            mean_absolute_error(y_test[:401565], lstm_test_predictions)
            ],
    'RMSE': [
              #math.sqrt(mean_squared_error(y_test, arima_predictions)),
             #math.sqrt(mean_squared_error(y_train, prophet_predictions)),
             math.sqrt(mean_squared_error(y_test[:401565], lstm_test_predictions))
             ],
    'Training Time': [
                      # arima_training_time
                      #, prophet_training_time,
                      lstm_training_time
                      ]
})

summary_df.to_csv(f'{save_path}/lstm_summary_2.csv', index=False)